In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("./data/properatti.csv", index_col=0)
data.shape

In [ ]:
# Seteamos pandas para no truncar los valores de las columnas
pd.set_option('display.max_colwidth', None)

In [ ]:
# Definimos dos grupos de columas para poder trabajar con ellas de forma mas sencilla
places = ['place_name','place_with_parent_names', 'country_name','state_name']
geolocation = ['geonames_id', 'lat-lon','lat','lon']

# Creamos un nuevo dataframe con las columnas que me interesan
location_data = data[places + geolocation].copy()
location_data.head()

### Exploración del dataset dividido en grupos de Properatti 

##### Ubicación

In [ ]:
places_data = data[places].copy()
places_data.head()

Exploración de las columnas relacionadas con la ubicación: Por Provincia, Ciudad/Barrio, el conjunto de ubicación 
- country_name
- state_name 
- place_name                        
- place_with_parent_names              
vamos a:
- Explorar las columnas 
- Visualizar los nan de estas columnas
- Corroborar que hay relación entre el ciudad/barrio o "place_name" y la concatenación de ubicación en "place_with_parent_names" para ver si efectivamente corresponden o hay barrios mezclados

- Exploramos las columnas 

In [ ]:
places_data.sample(10)

In [ ]:
#count: cuenta el numero de registros (no incluye null)
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna

places_data[places].describe()

In [ ]:
#ubicacion por provincia "State_name"
#notamos que los datos estan concentrados en Buenos Aires (Cap fed, zona norte, zona sur, oeste, atlantica, interior),
#  Cordoba, Santa fe 

places_data["state_name"].value_counts() #Excludes NA values by default.

In [ ]:
#ubicacion por ciudad-partido o barrio
 
places_data["place_name"].value_counts().head(20)

#Tigre es una ciudad al norte de la ciudad de Buenos Aires.
# Nordelta es una localidad urbana en el Partido de Tigre, Provincia de Buenos Aires 

##esta columna mezcla ciudades y barrios

#Capital Federal está como place_name, 

In [ ]:
#Vemos algunos registros de place_with_parent_names
places_data["place_with_parent_names"].value_counts().head(10) 

#el atributo representa el país/ "country_name", la provincia/ "state_name" (o division de la provincia), la ciudad principal/partido-municipio o barrio en el caso de capital federal"place_name"
#y un adicional que puede ser una localidad del partido (ejem: |Argentina|Bs.As. G.B.A. Zona Norte|Tigre|*Nordelta*|)
#el registro tiene entre 2  y 4 concatenaciones: 2 concatenaciones solo muestra hasta la ubicación por provincia, no brinda información relevante


###|Argentina|Capital Federal|  1297 no tiene nombre de barrio, desestimar o completar? 
#|Argentina|Córdoba|    2648

#esta columna puede usarse para completar los datos faltantes de place_name 
#podemos chequear si el atributo "place_name" coincide con los datos aquí plasmados

In [ ]:
#Para state_name vemos: cuales son los datos de place_name más frecuente, su frecuencia y cuantos registros diferentes existen 

places_data.groupby(["state_name"])["place_name"].describe()

#el atributo de place_name está representado principalmente por una ciudad-partido de la provincia

#count: cuenta el numero de registros no incluye null
#unique:  número de objetos distintos en la columna,
#top: es el dato más frecuente que se produce,
#freq: es la cantidad de veces que aparece el objeto “top” en la columna

- Visualizar los nan de las columnas
de aplicar la funcion isna().sum() en data obtuvimos
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0

#solo la columna place_name tiene valores nulos

In [ ]:
#vemos que filas son y como podemos completarlas o no

places_data[data['place_name'].isna()].head(2)

#los 23 datos pueden completarse con fillna con el dato de "Tigre" ya que vimos en place_name que "Tigre" 3324, un valor significativo


### Verificar la calidad de los datos:

Para verificar la calidad de los datos principalmente vamos a comparar las columnas que deberían tener la misma información y ver si coinciden o no. Por ejemplo, la columna "place_with_parent_names" debería contener dentro de su array la misma información que "place_name". Vamos a verificar si esto es cierto o no.

#### Corroborar si place_with_parent_names coincide con country_name, state_name, place_name 

In [ ]:
# Convertimos la columna place_with_parent_names en una lista usando el separador "|"
# Eliminamos el separador inicial y final de la lista
places_data['place_with_parent_names'] = places_data['place_with_parent_names'].apply(lambda x: x.lstrip("|").rstrip("|").split("|"))
places_data['place_with_parent_names'].head()

In [ ]:
# Función para chequear si los valores de la columna place_with_parent_names coinciden exactamente con 
# los valores de las columnas country_name, state_name, place_name
# Coincidir exactamente significa que la columna place_with_parent_names sólo tiene 3 elementos 
# y que los valores de los elementos coinciden en orden con los valores de las columnas country_name, state_name, place_name

def is_location_different(row):
    # la lista debería tener 3 elementos
    if len(row['place_with_parent_names']) != 3:
        return True
    if row['country_name'] == row['place_with_parent_names'][0] \
    and row['state_name'] == row['place_with_parent_names'][1] \
    and row['place_name'] == row['place_with_parent_names'][2]:
        return False
    else:
        return True

In [ ]:
# Creamos la mascara utilizando la función is_location_different
mask = places_data.apply(lambda x: is_location_different(x), axis=1)
print("place_with_parent_names difference with country_name, state_name and place_name:", places_data[mask].shape[0])
places_data[mask][places].sample(10)

*Conclusión: Existen 45220 registros que no coinciden exactamente.* 

### Análisis de los registros que no coinciden exactamente en las columnas places
___

Veamos cuáles son los registros que no coinciden exactamente en las columnas places

In [ ]:
# Contamos la cantidad de valores de cada fila de la lista place_with_parent_names
places_data['place_with_parent_names'].apply(lambda x: len(x)).value_counts()

Los que no coinciden exactamente son aquellas listas que no tienen 3 valores. Revisamos que los valores que están en cada lista tengan su contraparte en las otras columnas aunque figuren en otro orden del array. Por ejemplo, si en place_with_parent_names figura "Argentina|Capital Federal|Palermo" y en place_name figura "Capital Federal", entonces el registro para este momento del análisis se considera válido.

In [ ]:
# country_name
print("null values:", places_data['country_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['country_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with country_name:", places_data[mask].shape[0])

In [ ]:
# state_name
print("null values:", places_data['state_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['state_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with state_name:", places_data[mask].shape[0])

In [ ]:
# place_name
print("null values:", places_data['place_name'].isna().sum())
mask = ~places_data.apply(lambda x: x['place_name'] in x['place_with_parent_names'], axis=1)
print("place_with_parent_names difference with place_name:", places_data[mask].shape[0])


*Conclusión: La información que figura en las columnas place_name, country_name y state_name se encuentra en la columna place_with_parent_names aunque no exactamente igual (parece existir información extra) excepto en la columna 'place_name' con sus 23 NaNs que ya hemos identificado previamente.*

A continuación analizaremos esas columnas dividiendo el trabajo en partes dependiendo de la cantidad de elementos que contenga el array de la columna "place_with_parent_names".

#### Análisis de registros de place_with_parent_names con 3 valores
___

In [ ]:
# Revisamos los place_with_parent_names con tres elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 3)
places_data_3_elements = places_data[mask].copy()
places_data_3_elements.shape

In [ ]:
# Corroboramos que los valores de country_name, state_name y place_name coinciden con los valores de place_with_parent_names
print("country_name difference with place_with_parent_names[0]:", places_data_3_elements[places_data_3_elements['country_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[0])].shape[0])
print("state_name difference with place_with_parent_names[1]:", places_data_3_elements[places_data_3_elements['state_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[1])].shape[0])
print("place_name difference with place_with_parent_names[2]:", places_data_3_elements[places_data_3_elements['place_name'] != places_data_3_elements['place_with_parent_names'].apply(lambda x: x[2])].shape[0])

In [ ]:
# Vemos los datos correspondientes a los valores nulos de la columna place_name
mask = ~places_data.apply(lambda x: x['place_name'] in x['place_with_parent_names'], axis=1)
places_data_place_name_nan = places_data[mask].copy()
places_data_place_name_nan.head()

In [ ]:
places_data_place_name_nan['place_with_parent_names'].apply(lambda x: x[2]).value_counts()

*Conclusión: Los registros que no tienen valor en place_name son los que no coinciden, todos del municipio de Tigre. El resto de los valores son exactamente iguales*  





#### Análisis de registros de place_with_parent_names con 4 valores
___

In [ ]:
# Revisamos los place_with_parent_names con cuatro elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 4)
places_data_4_elements = places_data[mask].copy()
places_data_4_elements['place_with_parent_names'].shape


In [ ]:
# Contamos los valores de la columna place_with_parent_names por el segundo elemento (descartamos el primero porque sabemos que siempre es Argentina)
places_data_4_elements['place_with_parent_names'].apply(lambda x: x[1]).value_counts()


In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[1] coinciden con los valores de la columna state_name
mask = (places_data_4_elements['place_with_parent_names'].apply(lambda x: x[1]) != places_data_4_elements['state_name'])
print("place_with_parent_names[1] difference with state_name:", places_data_4_elements[mask].shape[0])


In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[2] coinciden con los valores de la columna place_name
mask = places_data_4_elements['place_with_parent_names'].apply(lambda x: x[2]) != places_data_4_elements['place_name']
print("place_with_parent_names[2] difference with place_name:", places_data_4_elements[mask].shape[0])
print("place_with_parent_names[2] proportional difference with place_name:", places_data_4_elements[mask].shape[0] / places_data_4_elements.shape[0])
places_data_4_elements_non_matching = places_data_4_elements[mask].copy()
places_data_4_elements_non_matching[places].sample(10)

In [ ]:
# No coinciden en 29842 registros pero coinciden en el resto. 
# Confirmamos que los que coinciden son iguales al valor[3]
mask = places_data_4_elements_non_matching['place_with_parent_names'].apply(lambda x: x[2]) == places_data_4_elements_non_matching['place_with_parent_names'].apply(lambda x: x[3])
places_data_4_elements_non_matching[mask].shape[0]/ places_data_4_elements_non_matching.shape[0]

In [ ]:
# Confirmamos que los valores de la columna place_with_parent_names[3] coinciden con los valores de la columna place_name
mask = (places_data_4_elements['place_with_parent_names'].apply(lambda x: x[3]) != places_data_4_elements['place_name'])
print("place_with_parent_names[3] difference with place_name:", places_data_4_elements[mask].shape[0])
anti_mask = ~mask # Los que coinciden
places_data_4_elements[anti_mask][places].sample(10)

*Conclusión: De los 39869 registros con 4 valores, 29842 toman el valor de place_name del 4to valor de la lista. Los 10027 restantes toman el 3er valor.*

- place_name asignado a 3er valor de "place_with_parent_names": 10027
- place_name asignado a 4to valor de "place_with_parent_names": 29842

#### Análisis de registros de place_with_parent_names con 2 valores
___

In [ ]:
# Reviso los place_with_parent_names con dos elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 2)
places_data_2_elements = places_data[mask].copy()
places_data_2_elements['place_with_parent_names'].value_counts()

In [ ]:
# Reviso los place_with_parent_names de Córdoba y Capital Federal
mask = places_data_2_elements['place_with_parent_names'].apply(lambda x: x[1] == 'Córdoba' or x[1] == 'Capital Federal')
places_data_2_elements[mask].sample(10)

In [ ]:
# Pareciera ser que todos los lugares que tienen dos elementos en place_with_parent_names repiten el nombre del estado en place_name
# Corroboramos esto
mask = places_data_2_elements['place_name'] != places_data_2_elements['state_name']
places_data_2_elements[mask].shape


*Conclusión: En los 4780 registros de place_with_parent_names que tienen 2 valores se utilizó state_name como place_name.*



#### Análisis de registros de place_with_parent_names con 5 valores
___

In [ ]:
# Reviso los place_with_parent_names con 5 elementos
mask = places_data['place_with_parent_names'].apply(lambda x: len(x) == 5)
places_data_5_elements = places_data[mask].copy()
places_data_5_elements['place_with_parent_names'].shape

In [ ]:
# Reviso si alguno de los valores no corresponde a Nordelta
places_data_5_elements['place_with_parent_names'].apply(lambda x: x[3] != 'Nordelta').sum()


In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[2] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[2]) != places_data_5_elements['place_name'])
print("place_with_parent_names[2] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[2] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])


In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[3] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[3]) != places_data_5_elements['place_name'])
print("place_with_parent_names[3] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[3] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])


In [ ]:
# Corroboramos que los valores de la columna place_with_parent_names[4] coinciden con los valores de la columna place_name
mask = (places_data_5_elements['place_with_parent_names'].apply(lambda x: x[4]) != places_data_5_elements['place_name'])
print("place_with_parent_names[4] difference with place_name:", places_data_5_elements[mask].shape[0])
print("place_with_parent_names[4] proportional difference with place_name:", places_data_5_elements[mask].shape[0] / places_data_5_elements.shape[0])

*Conclusión: Los 548 registros de place_with_parent_names que tienen 5 valores son de Nordelta y el último valor se refiere al Barrio. place_name toma los valores del barrio*


> ##### Conclusiones finales: 
> - Existen como máximo 5 tipos de registros en place_with_parent_names: Country, State, Municipio, Ciudad, Barrio
> - Country y State son iguales a country_name y state_name
> - Place_name se le asigna el valor de municipio: 76000 + 10027 = 86027
> - Place_name se le asigna el valor de ciudad: 29842 (en  estos casos existe el valor municipio también)
> - Place_name se le asigna el valor de barrio: 548 (en estos casos existe el valor municipio y ciudad también)
> - Place_name se le asigna el valor de state_name: 4780 (en estos casos no existe el valor municipio y ciudad)
> - Place_name se le asigna el valor de NaN: 23 (en estos casos existe el valor municipio y ciudad)
> ___
> Por lo tanto se puede concluir que para dar mayor consistencia es posible imputar place_name con el valor del municipio en las mayoría de los casos. Confrontar con geoNamesId.
> ___








## Imputación de valores de place_name
___